#### liste des fonctions à créer
- Échange
- Ballotage/Libérer joueur
- Commutations
    - Blessures
- Repêchages

In [8]:
#créer la table DRAFT en SQL avec les choix au repêchage
conn = sqlite3.connect('C:/Users/huber/OneDrive/NHL/DbMatchs.db')
choix = pd.read_csv("C:\dev\pool-de-hockey\src\data\listeChoix.csv", sep=";")
choix.to_sql('DRAFT', conn, if_exists='replace', index = False)

In [13]:
#créer une fausse table ALIGNEMENTS POUR FAIRE LES TESTS
alignements = pd.read_csv("alignements.csv", sep=";")
alignements.to_sql('copALIGN', conn, if_exists='replace', index = False)

In [20]:
# créer la fonction qui effectue l'échange pour un élément JOUEUR de l'échange

# la fonction doit faire deux choses:
        # 1. terminer l'ancienne période du joueur en changeant simplement dateFin
        # 2. ajouter la nouvelle période du joueur dans sa nouvelle équipe

trade = pd.read_csv("testTrade.csv", sep=";")
ids = trade["idNHL"].apply(str)

    #requête qui va chercher la ligne actuelle d'un joueur dans ALIGNEMENTS
df = []
for i in ids:
    requete = "SELECT * FROM copALIGN WHERE idNHL == " + i + " AND dateFin == '2100-01-01'"
    df = pd.read_sql(requete, conn)
df

,idAlignement,idPooler,prenomJoueur,nomJoueur,idNHL,statutJoueur,dateDebut,dateFin,pjActuels,butsActuels,assistActuels,pointsActuels
